In [ ]:
import json
import os
from datetime import date
from dotenv import load_dotenv
from openai import OpenAI
import httpx

load_dotenv()

LLAMA_STACK_URL = os.getenv("LLAMA_STACK_URL", "http://127.0.0.1:8321")
MODEL_ID = "vllm/openai/gpt-oss-20b"
MODEL_LABEL = "GPT-OSS-20b"
VLLM_VERSION = "0.11.2+rhai5"
RUN_DATE = os.getenv("RUN_DATE", date.today().isoformat())

# Set LLAMA_STACK_INSECURE=1 if you need to disable TLS verification
VERIFY_SSL = os.getenv("LLAMA_STACK_INSECURE", "0") != "1"

client = OpenAI(
    base_url=f"{LLAMA_STACK_URL}/v1",
    api_key="none",
    http_client=httpx.Client(verify=VERIFY_SSL)
)

PROMPTS = [
    "What were the top 3 headlines about NVIDIA in the last 7 days? Include sources.",
    "What is the latest stable release of vLLM and its release date? Cite sources.",
    "What is the most recent U.S. CPI (inflation) release and its month-over-month value? Cite sources."
]


def normalize_result(result):
    if isinstance(result, dict):
        return result
    if hasattr(result, "model_dump"):
        return result.model_dump()
    if hasattr(result, "__dict__"):
        return result.__dict__
    return {"value": str(result)}


def print_search_results(item):
    results = getattr(item, "results", None)
    if results is None:
        return
    print("    Results:")
    for idx, result in enumerate(results, 1):
        data = normalize_result(result)
        title = data.get("title")
        url = data.get("url")
        snippet = data.get("snippet") or data.get("content")
        print(f"      {idx}. {title}")
        if url:
            print(f"         URL: {url}")
        if snippet:
            print(f"         Snippet: {snippet}")


def run_prompt(prompt):
    print(f"\nUser: {prompt}")
    response = client.responses.create(
        model=MODEL_ID,
        tools=[{"type": "web_search"}],
        input=prompt
    )

    print(f"Response ID: {response.id}")
    search_triggered = False

    for item in response.output:
        item_type = getattr(item, "type", "unknown")
        print(f"  - Type: {item_type}")
        if "search" in item_type:
            search_triggered = True
        query = getattr(item, "query", None)
        if query:
            print(f"    Query: {query}")
        print_search_results(item)

    print(f"Search triggered: {search_triggered}")
    print("\n" + "=" * 60)
    print(f"Response from {MODEL_LABEL} with vLLM {VLLM_VERSION} ({RUN_DATE})")
    print("=" * 60)
    print(response.output_text)
    return response


for prompt in PROMPTS:
    run_prompt(prompt)


# Run Output (2026-01-19)

## Prompt 1

**Prompt:** What were the top 3 headlines about NVIDIA in the last 7 days? Include sources.

**Search triggered:** True

**Response status:** completed

**Response text:** (empty)

---

## Prompt 2

**Prompt:** What is the latest stable release of vLLM and its release date? Cite sources.

**Search triggered:** True

**Response status:** incomplete

**Response text:** (empty)

---

## Prompt 3

**Prompt:** What is the most recent U.S. CPI (inflation) release and its month-over-month value? Cite sources.

**Search triggered:** True

**Response status:** completed

**Response text:**

**Most recent U.S. CPI release (as of 13 January 2026)**  
- **Release date:** 13 January 2026 (for the month of December 2025)  
- **Month-over-month CPI change:** **+0.3%** (seasonally adjusted, CPI-U for all items)

**Source**

- Bureau of Labor Statistics, "Consumer Price Index – December 2025" (PDF)  
  - Key line: "The Consumer Price Index for All Urban Consumers (CPI-U) increased **0.3 percent** on a seasonally adjusted basis in December."  
  - https://www.bls.gov/news.release/pdf/cpi.pdf

---

## Full Response JSON (with metadata)

### Prompt 1

```json
{
  "id": "resp_0495bc07-485c-4a2d-b54c-aa15ff2785a8",
  "created_at": 1768833114.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": null,
  "model": "vllm/openai/gpt-oss-20b",
  "object": "response",
  "output": [
    {
      "id": "fc_a80e052f-1298-409c-81fa-cc35a6f7d922",
      "action": null,
      "status": "completed",
      "type": "web_search_call"
    },
    {
      "id": "fc_acf72863-2e5d-4812-840e-f213643cca65",
      "action": null,
      "status": "completed",
      "type": "web_search_call"
    },
    {
      "id": "fc_15917666-e04a-44ab-b17f-ced42d29a38a",
      "action": null,
      "status": "completed",
      "type": "web_search_call"
    },
    {
      "id": "fc_b95c9bcc-77bd-4932-9b0b-d79cd38ca39a",
      "action": null,
      "status": "completed",
      "type": "web_search_call"
    },
    {
      "id": "msg_f02e3c59-79bb-4b64-993e-8b52ca22eb9c",
      "content": [
        {
          "annotations": [],
          "text": "",
          "type": "output_text",
          "logprobs": null
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type": "message"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": null,
  "tool_choice": null,
  "tools": [
    {
      "type": "web_search",
      "filters": null,
      "search_context_size": "medium",
      "user_location": null
    }
  ],
  "top_p": null,
  "background": null,
  "conversation": null,
  "max_output_tokens": null,
  "max_tool_calls": null,
  "previous_response_id": null,
  "prompt": null,
  "prompt_cache_key": null,
  "reasoning": null,
  "safety_identifier": null,
  "service_tier": null,
  "status": "completed",
  "text": {
    "format": {
      "type": "text"
    },
    "verbosity": null
  },
  "top_logprobs": null,
  "truncation": null,
  "usage": {
    "input_tokens": 23288,
    "input_tokens_details": null,
    "output_tokens": 666,
    "output_tokens_details": null,
    "total_tokens": 23954
  },
  "user": null
}
```

### Prompt 2

```json
{
  "id": "resp_a6f1d5bd-379a-437c-bc9a-cf4dd308011d",
  "created_at": 1768833133.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": null,
  "model": "vllm/openai/gpt-oss-20b",
  "object": "response",
  "output": [
    {
      "id": "fc_920079c4-11f6-4c81-a19b-292ab2837619",
      "action": null,
      "status": "completed",
      "type": "web_search_call"
    },
    {
      "id": "fc_b0d0be3e-1519-49a0-a3d0-6e9b39fd0b74",
      "action": null,
      "status": "completed",
      "type": "web_search_call"
    },
    {
      "id": "fc_f1e0de3c-7389-49c6-858b-54bd3dd5de5d",
      "action": null,
      "status": "completed",
      "type": "web_search_call"
    },
    {
      "id": "fc_ee1743be-4bc9-480b-a1e0-837457e5afbb",
      "action": null,
      "status": "completed",
      "type": "web_search_call"
    },
    {
      "id": "fc_e411e46b-0028-400e-aae5-a11cfa646d55",
      "action": null,
      "status": "completed",
      "type": "web_search_call"
    },
    {
      "id": "fc_3bf91bfd-992e-4fef-88e0-6aeda8d0242a",
      "action": null,
      "status": "completed",
      "type": "web_search_call"
    },
    {
      "id": "fc_3a090d85-c625-4ed9-9a78-f8f549e2165f",
      "action": null,
      "status": "completed",
      "type": "web_search_call"
    },
    {
      "id": "fc_84996674-e4bf-4e38-8bcf-1f9ff292f712",
      "action": null,
      "status": "completed",
      "type": "web_search_call"
    },
    {
      "id": "fc_e20c9bde-5b11-4172-a9d2-4bea97fe6782",
      "action": null,
      "status": "completed",
      "type": "web_search_call"
    },
    {
      "id": "fc_9ab92be4-f7cf-494a-8dbc-55a1f5c9bdd6",
      "action": null,
      "status": "completed",
      "type": "web_search_call"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": null,
  "tool_choice": null,
  "tools": [
    {
      "type": "web_search",
      "filters": null,
      "search_context_size": "medium",
      "user_location": null
    }
  ],
  "top_p": null,
  "background": null,
  "conversation": null,
  "max_output_tokens": null,
  "max_tool_calls": null,
  "previous_response_id": null,
  "prompt": null,
  "prompt_cache_key": null,
  "reasoning": null,
  "safety_identifier": null,
  "service_tier": null,
  "status": "incomplete",
  "text": {
    "format": {
      "type": "text"
    },
    "verbosity": null
  },
  "top_logprobs": null,
  "truncation": null,
  "usage": {
    "input_tokens": 54789,
    "input_tokens_details": null,
    "output_tokens": 1047,
    "output_tokens_details": null,
    "total_tokens": 55836
  },
  "user": null
}
```

### Prompt 3

```json
{
  "id": "resp_8880af73-6b0d-4489-b7f1-9712b7251733",
  "created_at": 1768833167.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": null,
  "model": "vllm/openai/gpt-oss-20b",
  "object": "response",
  "output": [
    {
      "id": "fc_67f41ac5-1fcb-4d21-b7a9-9d138ee8e733",
      "action": null,
      "status": "completed",
      "type": "web_search_call"
    },
    {
      "id": "fc_12ccc4c2-0f60-4e1b-b374-309034b5c76d",
      "action": null,
      "status": "completed",
      "type": "web_search_call"
    },
    {
      "id": "msg_80d2e514-7027-4523-b940-84cd9945224e",
      "content": [
        {
          "annotations": [],
          "text": "**Most recent U.S. CPI release (as of 13\u202fJanuary\u202f2026)**  \n- **Release date:** 13\u202fJanuary\u202f2026 (for the month of December\u202f2025)  \n- **Month\u2011over\u2011month CPI change:** **+0.3\u202f%** (seasonally adjusted, seasonally\u2011adjusted CPI\u2011U for all items)\n\n**Source**\n\n- Bureau of Labor Statistics, \u201cConsumer Price Index \u2013 December\u202f2025\u201d (PDF)  \n  - *Key line:* \u201cThe Consumer Price Index for All Urban Consumers (CPI\u2011U) increased **0.3 percent** on a seasonally adjusted basis in December.\u201d  \n  - Link: https://www.bls.gov/news.release/pdf/cpi.pdf",
          "type": "output_text",
          "logprobs": null
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type": "message"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": null,
  "tool_choice": null,
  "tools": [
    {
      "type": "web_search",
      "filters": null,
      "search_context_size": "medium",
      "user_location": null
    }
  ],
  "top_p": null,
  "background": null,
  "conversation": null,
  "max_output_tokens": null,
  "max_tool_calls": null,
  "previous_response_id": null,
  "prompt": null,
  "prompt_cache_key": null,
  "reasoning": null,
  "safety_identifier": null,
  "service_tier": null,
  "status": "completed",
  "text": {
    "format": {
      "type": "text"
    },
    "verbosity": null
  },
  "top_logprobs": null,
  "truncation": null,
  "usage": {
    "input_tokens": 8380,
    "input_tokens_details": null,
    "output_tokens": 517,
    "output_tokens_details": null,
    "total_tokens": 8897
  },
  "user": null
}
```
